# FeatureExtractionClassifier

Use various features extracted from the WAV file to classify genres.
Architecture implementation based on features derived at: https://www.kaggle.com/code/dramirdatascience/gtzan-music-classification-using-ml-acc-93-24

In [ ]:
!pip install "ray[tune]"
import torch
import torch.nn as nn
import numpy as np
from utils import *
import torchvision.transforms as transforms
import torch.utils.data as Data
from scipy.io import wavfile
from ray import air
import os
from ray.tune.schedulers import ASHAScheduler
import csv

## Mount drive
Mount google drive if running on google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Constant parameters used in training

Run `setup.sh` to mount Google Drive containing GTZAN

In [ ]:
GENRES = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3,
          'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8,
          'rock': 9}

GTZAN_CSV = "/content/drive/MyDrive/GTZAN/Data/features_3_sec.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", DEVICE)

## Training

Create a `Dataset` for the audio files

Obtain the features we are intested in from the WAV file. There are a total of 57 distinct features that are provided in the CSVs attached along with GTZAN. Using my understanding from [This analysis by the dataset author](https://www.kaggle.com/code/dramirdatascience/gtzan-music-classification-using-ml-acc-93-24), I have described the use of each of those features and how they were obtained in brief. For more in-depth understanding of how the features were obtained, the author has attached source code as well.

We split each song into 10 parts as the features we extract are relevant only within a short time-frame, such as features regarding pitch/frequency.

### Chroma STFT

The same principle of using STFT to extract frequency magnitudes accross time-intervals as used in `mel_spec_classifier`. However, the frequency classes used now match the 12 pitch classes in western music instead. In effect, we end up looking at far less data and specifically the 12 pitches that are most useful when trying to understand genre.

We obtain the mean and variance w.r.t magnitude to determine the base pitch of the song.

### MFCC

Again, same principle of using STFT to extract frequency magnitudes accross time-intervals as used in `mel_spec_classifier`. We don't  just analyze the spectogram here, but instead obtain the mean and variance of frequencies to again determine features regarding the frequency of the song.

### RMS
Average power of the song. Not unreasonable to theorize that Metal/Rock would be 'louder' on average.

### Spectral Centroid Mean
'Centre of mass' of frequencies in a time interval. Understanding of 'brightness' of a song, as higher C.O.M -> higher frequency -> perceived brightness.

### Spectral Bandwith Mean
Provides information on mean spread of frequencies.

### Spectral Rolloff Mean
Another metric of audio signal bandwidth. Gives frequency bin under which 50% of total energy exists.

### Tempo Mean

Tempo Mean calculates avg. perceived tempo by beat tracking, which essentailly measures power overtime and deterining BPM from the periodicity.

### Zero Crossing Rate Mean

Average rate that audio signal crosses zero axis over time. Tells us about rapid changes in audio signal

### MFCC

The Mel Spec



In [ ]:
class FeatureExtractionDataset(Data.Dataset):
  def __init__(self):
    self.data = []
    self.labels = []

    # Go through all songs and tag X (tensor of features), Y as genre.
    with open(GTZAN_CSV, newline='') as csvfile:
      reader = csv.reader(csvfile)
      for row in reader:
        self.labels.append(GENRES[row[-1]])
        # Contains all the features (comma seperated) as mentioned above
        self.data.append(torch.tensor(row[2:-1], dtype=torch.float32))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx]

With the features, two fully connected layers is the preliminary model that we use. We do not fase a dimensionality issue as we did in the previous cases (`mel_spec` and `wav` classifiers)

In [ ]:
class FeatureExtractionClassifier(nn.Module):
  def __init__(self, l1=256, l2 = 20):
    super().__init__()

    # 57 input nodes as there are 57 features extracted.
    self.fcs = nn.Sequential(
        nn.Linear(57, l1),
        nn.ReLU(),
        nn.Linear(l1, l2),
        nn.ReLU())

  def forward(self, x):
    return self.fcs(x)

Create routines for training and validation. Perform Hyperparameter Tuning to devise a closer to optimized model.

In [ ]:
def train_feature_extraction_model(config):

  model = FeatureExtractionClassifier(l1=config["l1"], l2=config["l2"])
  model.to(DEVICE)
  feature_extraction_dataset = FeatureExtractionDataset()

  # train model with training dataset, but ray tuner uses validation dataset to tune hyperparameters
    train_model(model, DEVICE, config, feature_extraction_dataset)

## Testing

 Create routine for testing model. The split being used is 80% for training, 10% for validation, and 10% for testing.

In [ ]:
def test_feature_extraction_model(best_result):
  best_model = FeatureExtractionClassifier(l1=best_result["l1"], l2=best_result["l2"])
  best_model.to(DEVICE)

  feature_extraction_dataset = FeatureExtractionDataset()
  test_model(best_model, best_result, feature_extraction_dataset, DEVICE)